In [ ]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

# Step 1: Read the Excel file
data_file = r"file_path"
data = pd.read_excel(data_file)

# Ensure the column names in the Excel file match the database table structure
# For example, check if columns are ['last_name', 'first_name', 'crop_name', 'variety', 'initial_planting_year', 'tree_spacing', 'tree_flower_percentage', 'flowering_image_url', 'avg_fruit_set_percentage', 'avg_fruit_count', 'predicted_harvest_time']

# Step 2: Establish connection to the MySQL database
try:
    connection = mysql.connector.connect(
        host='localhost',  # Change to your host
        user='root',  # Replace with your MySQL username
        password='******',  # Replace with your MySQL password
        database='*****'  # Replace with your database name
    )

    if connection.is_connected():
        cursor = connection.cursor(dictionary=True)

        # Step 3: Retrieve farmer and farm IDs based on last_name, first_name, and farm_number
        for index, row in data.iterrows():
            last_name = row['last_name']
            first_name = row['first_name']
            farm_number = row['farm_number']

            # Query to get farmer_id
            cursor.execute(
                """
                SELECT farmer_id FROM farmer WHERE last_name = %s AND first_name = %s
                """, (last_name, first_name)
            )
            farmer_result = cursor.fetchone()

            if farmer_result:
                farmer_id = farmer_result['farmer_id']

                # Ensure all previous results are read before executing a new query
                cursor.fetchall()

                # Query to get farm_id
                cursor.execute(
                    """
                    SELECT farm_id FROM farm WHERE farm_number = %s AND farmer_id = %s
                    """, (farm_number, farmer_id)
                )
                farm_result = cursor.fetchone()

                if farm_result:
                    farm_id = farm_result['farm_id']

                    # Ensure all previous results are read before executing a new query
                    cursor.fetchall()

                    # Step 4: Prepare the insert statement
                    insert_query = """
                    INSERT INTO main_crop_data (
                        farmer_id, farm_id, crop_name, variety, initial_planting_year, tree_spacing,
                        tree_flower_percentage, flowering_image_url, avg_fruit_set_percentage,
                        avg_fruit_count, predicted_harvest_time
                    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                    """

                    cursor.execute(
                        insert_query,
                        (
                            farmer_id,
                            farm_id,
                            row['crop_name'] if pd.notna(row['crop_name']) else None,
                            row['variety'] if pd.notna(row['variety']) else None,
                            row['initial_planting_year'] if pd.notna(row['initial_planting_year']) else None,
                            row['tree_spacing'] if pd.notna(row['tree_spacing']) else None,
                            row['tree_flower_percentage'] if pd.notna(row['tree_flower_percentage']) else None,
                            row['flowering_image_url'] if pd.notna(row['flowering_image_url']) else None,
                            row['avg_fruit_set_percentage'] if pd.notna(row['avg_fruit_set_percentage']) else None,
                            row['avg_fruit_count'] if pd.notna(row['avg_fruit_count']) else None,
                            row['predicted_harvest_time'] if pd.notna(row['predicted_harvest_time']) else None
                        )
                    )

        # Step 5: Commit the transaction
        connection.commit()
        print("Data imported successfully!")

except Error as e:
    print(f"Error: {e}")

finally:
    # Step 6: Close the connection
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection closed.")
